In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import datetime

In [3]:
rm -rf ./log/

In [4]:
mnist = tf.keras.datasets.mnist

(train, train_label), (test, test_label) = mnist.load_data()
train, test = train / 255.0, test / 255.0

def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation="softmax")
    ])

In [5]:
model = create_model()
model.compile(optimizer="adam",
             loss="sparse_categorical_crossentropy",
             metrics=['acc'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=train,
         y=train_label,
         epochs=5,
         validation_data=(test, test_label),
         callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2188 - acc: 0.9353 - val_loss: 0.0988 - val_acc: 0.9701
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0953 - acc: 0.9707 - val_loss: 0.0798 - val_acc: 0.9756
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0701 - acc: 0.9777 - val_loss: 0.0654 - val_acc: 0.9795
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0541 - acc: 0.9827 - val_loss: 0.0732 - val_acc: 0.9768
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0444 - acc: 0.9849 - val_loss: 0.0781 - val_acc: 0.9772


In [6]:
%tensorboard --logdir logs/fit

# 다른 방법으로 텐서 보드 사용하기

In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((train, train_label))
test_dataset = tf.data.Dataset.from_tensor_slices((test, test_label))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [8]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [9]:
train_loss = tf.keras.metrics.Mean("train_loss", dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy("train_accuracy")
test_loss = tf.keras.metrics.Mean("test_loss", dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy("test_accurcay")

In [10]:
def train_step(model, optimizer, train, train_label):
    with tf.GradientTape() as tape:
        predictions = model(train, training=True)
        loss = loss_object(train_label, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(train_label, predictions)
    
def test_step(model, test, test_label):
    predictions = model(test)
    loss = loss_object(test_label, predictions)
    
    test_loss(loss)
    test_accuracy(test_label, predictions)

In [11]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = "logs/gradient_tape/" + current_time + "/train"
test_log_dir = "logs/gradient_tape/" + current_time + "/test"
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [12]:
model = create_model()

EPOCHS = 5

for epoch in range(EPOCHS):
    for (train, train_label) in train_dataset:
        train_step(model, optimizer, train, train_label)
    with train_summary_writer.as_default():
        tf.summary.scalar("loss", train_loss.result(), step=epoch)
        tf.summary.scalar("accuracy", test_accuracy.result(), step=epoch)
    
    for (test, test_label) in test_dataset:
        test_step(model, test, test_label)
    with train_summary_writer.as_default():
        tf.summary.scalar("loss", test_loss.result(), step=epoch)
        tf.summary.scalar("accuracy", test_accuracy.result(), step=epoch)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))
    
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.24856920540332794, Accuracy: 92.75166320800781, Test Loss: 0.12042733281850815, Test Accuracy: 96.29000091552734
Epoch 2, Loss: 0.10552313178777695, Accuracy: 96.81999969482422, Test Loss: 0.08581424504518509, Test Accuracy: 97.53999328613281
Epoch 3, Loss: 0.07286021113395691, Accuracy: 97.79499816894531, Test Loss: 0.07056759297847748, Test Accuracy: 97.75999450683594
Epoch 4, Loss: 0.055525217205286026, Accuracy: 98.29166412353516, Test Loss: 0.06444180756807327, Test Accuracy: 97.90999603271484
Epoch 5, Loss: 0.04488527402281761, Accuracy: 98.61833190917969, Test Loss: 0.06335631757974625, Test Accuracy: 97.83999633789062


In [13]:
%tensorboard --logdir logs/gradient_tape